<a href="https://colab.research.google.com/github/jhtwiz/AI-1-jhtwiz/blob/main/2%EC%A3%BC%EC%B0%A8_%EC%8B%AC%ED%99%94%EA%B3%BC%EC%A0%9C_%EC%B5%9C%EC%A2%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformer 실습

이번 실습에서는 감정 분석 task에 RNN 대신 Transformer를 구현하여 적용해 볼 것입니다.
Library import나 dataloader 생성은 RNN 실습 때와 똑같기 때문에 설명은 넘어가도록 하겠습니다.

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [3]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 17.0 MB/s eta 0:00:00


In [4]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import BertTokenizerFast
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)


ds = load_dataset("stanfordnlp/imdb")
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')


def new_collate_fn(batch):
  max_len = 400
  texts, labels = [], []

  for row in batch:
    tokens = tokenizer(row['text'], truncation=True, max_length=max_len).input_ids
    labels.append(tokens[-2]) # 토큰화 하게 되면 [CLS], ... , 마지막 token_id,[SEP] 일거니 마지막 token_id 추출
    texts.append(tokens[:-2]) # labels 앞까지 추출.

  # (B, S)에서 S를 동일하게 맞추기 위해 가장 긴 Sequence를 기준으로 padding 추가
  texts = pad_sequence([torch.LongTensor(text) for text in texts], batch_first=True, padding_value=tokenizer.pad_token_id)
  labels = torch.LongTensor(labels)

  return texts, labels


new_train_loader = DataLoader(
    ds['train'], batch_size=64, shuffle=True, collate_fn=new_collate_fn
)
new_test_loader = DataLoader(
    ds['test'], batch_size=64, shuffle=False, collate_fn=new_collate_fn
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
dataiter = iter(new_train_loader)
texts, labels = next(dataiter)
print(tokenizer.convert_ids_to_tokens(labels))
print(labels)

['.', '.', 'solely', '.', 'car', '/', '.', ',', '.', '*', '.', '>', '.', 'of', '.', 'joseph', '.', '.', '>', '.', '.', '.', 'and', 'story', '.', '.', 'c', 'lee', 'do', '!', '.', '.', '.', '.', '!', 'is', '.', '.', 'the', '.', '.', '!', '.', '.', 'attempted', '.', '?', 'very', 'the', '.', '.', '.', '.', '10', '.', '.', '.', '.', '.', '*', 'jane', '.', '.', '.']
tensor([1012, 1012, 9578, 1012, 2482, 1013, 1012, 1010, 1012, 1008, 1012, 1028,
        1012, 1997, 1012, 3312, 1012, 1012, 1028, 1012, 1012, 1012, 1998, 2466,
        1012, 1012, 1039, 3389, 2079,  999, 1012, 1012, 1012, 1012,  999, 2003,
        1012, 1012, 1996, 1012, 1012,  999, 1012, 1012, 4692, 1012, 1029, 2200,
        1996, 1012, 1012, 1012, 1012, 2184, 1012, 1012, 1012, 1012, 1012, 1008,
        4869, 1012, 1012, 1012])


In [6]:
from torch import nn
from math import sqrt


class MultiHeadAttention(nn.Module):
  def __init__(self, input_dim, d_model, n_heads):
    super().__init__()
    assert d_model % n_heads == 0

    self.input_dim = input_dim
    self.d_model = d_model # D
    self.n_heads = n_heads # H
    self.head_dim = d_model // n_heads # D/H


    self.wq = nn.Linear(input_dim, d_model)
    self.wk = nn.Linear(input_dim, d_model)
    self.wv = nn.Linear(input_dim, d_model)
    self.dense = nn.Linear(d_model, d_model)

    self.softmax = nn.Softmax(dim=-1)

  def forward(self, x, mask):
    batch_size = x.shape[0] # B
    seq_length = x.shape[1] # S

    q, k, v = self.wq(x), self.wk(x), self.wv(x) # (B, S, D)
    q = q.view(batch_size, seq_length, self.n_heads, self.head_dim).transpose(1, 2) # (B, S, D) -> (B, S, H, D/H) -> (B, H, S, D/H)
    k = k.view(batch_size, seq_length, self.n_heads, self.head_dim).transpose(1, 2) # (B, S, D) -> (B, S, H, D/H) -> (B, H, S, D/H)
    v = v.view(batch_size, seq_length, self.n_heads, self.head_dim).transpose(1, 2) # (B, S, D) -> (B, S, H, D/H) -> (B, H, S, D/H)
    score = torch.matmul(q, k.transpose(-1, -2)) # (B, H, S, D/H) * (B, H, D/H, S) = (B, H, S, S)
    score = score / sqrt(self.head_dim) # sqrt(D/H)

    if mask is not None:
      # mask 차원 추가는 TextClassifier에 적용했습니다
      score = score + (mask * -1e9)

    score = self.softmax(score)
    result = torch.matmul(score, v) # (B, H, S, S) * (B, H, S, D/H) = (B, H, S, D/H)
    result = result.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model) # (B, H, S, D/H) -> (B, S, D)
    result = self.dense(result)

    return result

In [7]:
class TransformerLayer(nn.Module):
  def __init__(self, input_dim, d_model, n_heads, dff, p_dropout):
    super().__init__()

    self.input_dim = input_dim
    self.d_model = d_model
    self.dff = dff

    self.MHA = MultiHeadAttention(input_dim, d_model, n_heads)
    self.FFN = nn.Sequential(
      nn.Linear(d_model, dff),
      nn.ReLU(),
      nn.Linear(dff, d_model)
    )
    self.dropout = nn.Dropout(p_dropout)
    self.layerNorm1 = nn.LayerNorm(d_model)
    self.layerNorm2 = nn.LayerNorm(d_model)

  def forward(self, x, mask):
    MHA = self.dropout(self.MHA(x, mask)) # MHA with dropout
    x = self.layerNorm1(x + MHA) # residual connection(Add+Norm)

    FFN = self.dropout(self.FFN(x)) # FFN with dropout
    x = self.layerNorm2(x + FFN) # residual connection(Add+Norm)

    return x

In [8]:
import numpy as np


def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, None], np.arange(d_model)[None, :], d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[None, ...]

    return torch.FloatTensor(pos_encoding)


max_len = 400
print(positional_encoding(max_len, 256).shape)

torch.Size([1, 400, 256])


In [35]:
class NewTextClassifier(nn.Module):
  def __init__(self, vocab_size, d_model, n_heads, n_layers, dff, p_dropout = 0.1):
    super().__init__()

    self.vocab_size = vocab_size
    self.d_model = d_model
    self.n_layers = n_layers
    self.dff = dff

    self.embedding = nn.Embedding(vocab_size, d_model)
    self.pos_encoding = nn.parameter.Parameter(positional_encoding(max_len, d_model), requires_grad=False)
    self.layers = nn.ModuleList([TransformerLayer(d_model, d_model, n_heads, dff, p_dropout) for _ in range(n_layers)])
    self.classification = nn.Linear(d_model, vocab_size)

  def forward(self, x):
    mask = (x == tokenizer.pad_token_id)
    mask = mask[:, None, None, :]
    batch_size = x.shape[0]
    seq_len = x.shape[1]

    x = self.embedding(x)
    x = x * sqrt(self.d_model)
    x = x + self.pos_encoding[:, :seq_len]

    for layer in self.layers:
      x = layer(x, mask)

    # padding이 아닌 부분 찾기
    non_pad_mask = (mask[:,0,0,:] == False)
    valid_seq_index = non_pad_mask.sum(dim=1) - 1 # 각 시퀀스에서 마지막 실제 단어의 index

    # 배치의 각 시퀀스에서 마지막 단어 위치에 있는 임베딩 벡터를 추출
    # 이전의 x[:, -1]은 모든 sequence들의 마지막 것만 갖고 왔는데 padding이 잡히는걸 방지하기 위해 아래 방법 시도
    x = x[torch.arange(batch_size), valid_seq_index]  # [batch_size, d_model]

    x = self.classification(x)

    return x


In [36]:
from torch.optim import Adam

lr = 0.001

model_new = NewTextClassifier(len(tokenizer), 128, 4, 5, 256) # 5-layer-4-head, 기본과제에서 가장 성능이 좋았던 d_model=128, dff=256로
model_new = model_new.to('cuda')
loss_fn = nn.CrossEntropyLoss()

optimizer_new = Adam(model_new.parameters(), lr=lr)

In [23]:
import numpy as np
import matplotlib.pyplot as plt


def accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for data in dataloader:
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda')

    preds = model(inputs)
    preds = torch.argmax(preds, dim=-1)

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt

In [24]:
n_epochs = 15

for epoch in range(n_epochs):
  total_loss = 0.
  model_new.train()
  for data in new_train_loader:
    model_new.zero_grad()
    inputs, labels = data
    inputs, labels = inputs.to('cuda').long(), labels.to('cuda').long()

    preds = model_new(inputs)
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer_new.step()

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

  with torch.no_grad():
    model_new.eval()
    train_acc = accuracy(model_new, new_train_loader)
    test_acc = accuracy(model_new, new_test_loader)
    print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

Epoch   0 | Train Loss: 1381.2168748378754
=========> Train acc: 0.558 | Test acc: 0.564
Epoch   1 | Train Loss: 1119.6556376218796
=========> Train acc: 0.608 | Test acc: 0.611
Epoch   2 | Train Loss: 965.2885994911194
=========> Train acc: 0.616 | Test acc: 0.622
Epoch   3 | Train Loss: 886.6888453960419
=========> Train acc: 0.630 | Test acc: 0.629
Epoch   4 | Train Loss: 820.8058145046234
=========> Train acc: 0.635 | Test acc: 0.627
Epoch   5 | Train Loss: 745.7131525278091
=========> Train acc: 0.644 | Test acc: 0.621
Epoch   6 | Train Loss: 667.5163964033127
=========> Train acc: 0.672 | Test acc: 0.626
Epoch   7 | Train Loss: 589.178312420845
=========> Train acc: 0.706 | Test acc: 0.623
Epoch   8 | Train Loss: 521.8718618154526
=========> Train acc: 0.731 | Test acc: 0.617
Epoch   9 | Train Loss: 464.5834765434265
=========> Train acc: 0.745 | Test acc: 0.620
Epoch  10 | Train Loss: 422.89216578006744
=========> Train acc: 0.774 | Test acc: 0.607
Epoch  11 | Train Loss: 388.80

**train 데이터의 정확도는 올라가지만 test데이터는 60~63%사이를 정체중. 이 이상은 돌려도 무의미할듯**

**학습한 문장은 학습한대로 단어를 잘 예측하지만 새로운 문장에 대해서는 잘 예측하지 못하는 듯**

In [25]:
def accuracyWithPrint(model, dataloader):
  cnt = 0
  acc = 0

  for i, data in enumerate(dataloader):
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda')

    preds = model(inputs)
    preds = torch.argmax(preds, dim=-1)
    if i % 100 == 0:
      print(f'{"="*50}predict{"="*50}')
      print(tokenizer.convert_ids_to_tokens(preds))
      print(preds)
      print(f'{"="*50}label{"="*50}')
      print(tokenizer.convert_ids_to_tokens(labels))
      print(labels)
      print('Is Correct?')
      print(labels == preds)
      print(f'{(labels == preds).sum().item()} / 64')

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt

In [26]:
with torch.no_grad():
    model_new.eval()
    result = accuracyWithPrint(model_new, new_test_loader)
    print(f"=========> Acc: {result:.3f}")

==================================================predict==================================================
['.', '.', '.', 'such', '.', '.', 'only', '.', '!', '.', '.', '.', '.', 't', '.', '.', '.', '.', '.', '.', '.', 'an', '.', 'the', '.', '.', 'want', 'movie', 'watch', '>', '.', '.', 'bench', '.', '.', '.', 't', '.', '*', '.', '.', ')', '.', '.', '.', '.', 'work', ')', '.', '.', 'and', 'creatures', '.', '*', 'evil', 'fulfilling', 'off', '##ge', '.', '"', ',', '.', '!', '10']
tensor([ 1012,  1012,  1012,  2107,  1012,  1012,  2069,  1012,   999,  1012,
         1012,  1012,  1012,  1056,  1012,  1012,  1012,  1012,  1012,  1012,
         1012,  2019,  1012,  1996,  1012,  1012,  2215,  3185,  3422,  1028,
         1012,  1012,  6847,  1012,  1012,  1012,  1056,  1012,  1008,  1012,
         1012,  1007,  1012,  1012,  1012,  1012,  2147,  1007,  1012,  1012,
         1998,  7329,  1012,  1008,  4763, 21570,  2125,  3351,  1012,  1000,
         1010,  1012,   999,  2184], device='cud

In [27]:
#혹시 모르니 5 epoch만 더
n_epochs = 5

for epoch in range(n_epochs):
  total_loss = 0.
  model_new.train()
  for data in new_train_loader:
    model_new.zero_grad()
    inputs, labels = data
    inputs, labels = inputs.to('cuda').long(), labels.to('cuda').long()

    preds = model_new(inputs)
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer_new.step()

    total_loss += loss.item()

  print(f"Epoch {15+epoch:3d} | Train Loss: {total_loss}")

  with torch.no_grad():
    model_new.eval()
    train_acc = accuracy(model_new, new_train_loader)
    test_acc = accuracy(model_new, new_test_loader)
    print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

Epoch  15 | Train Loss: 261.50021597743034
=========> Train acc: 0.877 | Test acc: 0.601
Epoch  16 | Train Loss: 230.1661384999752
=========> Train acc: 0.880 | Test acc: 0.591
Epoch  17 | Train Loss: 205.95276042819023
=========> Train acc: 0.907 | Test acc: 0.603
Epoch  18 | Train Loss: 182.2091498374939
=========> Train acc: 0.911 | Test acc: 0.604
Epoch  19 | Train Loss: 170.065526381135
=========> Train acc: 0.925 | Test acc: 0.590


역시 overfitting이 심화되고 있음